In [1]:
import csv
import psycopg2
import re
import spacy

# Database connection parameters
db_params = {
    "dbname": "authors_db",
    "user": "book_user",
    "password": "@book24**",
    "host": "book_host",
    "port": "5432",
}

# change the db parameters of your own. this is are individual test parameter credentials.
# Load the spaCy model for English language processing
nlp = spacy.load("en_core_web_sm")

# Constants
CHUNK_SIZE = 50
MAX_AUTHORS = 250
MAX_TO_READ = 500
MIN_AUTHORS_REQUIRED = 250

def clean_title(title):
    """Cleans and validates book titles."""
    title = re.sub(r'^[“"”‘’]+|[“"”‘’]+$', "", title.strip())
    meaningless_words = {"phd", "ka", "the", "a", "an", "in", "on", "of", "for", "and"}
    words = title.split()
    
    if len(words) < 2 or len(words) > 5 or not set(word.lower() for word in words).intersection(meaningless_words):
        return title.strip()
    return None

def is_english(title):
    """Checks if the title is in English."""
    return all(word.isalpha() or word.isspace() for word in title.split())

def extract_book_titles(description):
    """Extracts book titles from the author's description using spaCy."""
    doc = nlp(description)
    titles = [clean_title(ent.text) for ent in doc.ents if ent.label_ == "WORK_OF_ART"]
    return [title for title in titles if title and is_english(title)]

def construct_bio(about_text):
    """Returns the original bio text without modification."""
    return about_text


def validate_author_data(author):
    """Validates the author's data."""
    return all([author["name"], author["image_url"], author["about"]])

def read_authors_from_csv(file_path):
    """Reads authors from a CSV file and returns a list of dictionaries."""
    authors_data = []
    try:
        with open(file_path, mode="r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                if len(authors_data) >= MAX_TO_READ:
                    break

                name = row.get("name", "").strip()
                image_url = row.get("image_url", "").strip()
                about = row.get("about", "").strip()

                if name and image_url and about:
                    about_clean = re.sub(r"<.*?>", "", about)
                    authors_data.append({
                        "name": name,
                        "image_url": image_url,
                        "about": about_clean,  # Keep the cleaned bio intact
                        "books_written": extract_book_titles(about_clean),
                    })


        print(f"Read {len(authors_data)} authors from {file_path}.")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred while reading CSV: {e}")

    return authors_data

def insert_data_in_chunks(authors_data):
    """Inserts authors and their books into the database in chunks."""
    valid_authors_count = 0
    try:
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                for i in range(0, len(authors_data), CHUNK_SIZE):
                    chunk = authors_data[i: i + CHUNK_SIZE]
                    author_ids = {}

                    for author in chunk:
                        if not validate_author_data(author):
                            print(f"Invalid author data for: {author['name']}. Skipping.")
                            continue

                        author_id = insert_author(cur, author)
                        if author_id:
                            author_ids[author["name"]] = author_id
                            valid_authors_count += 1

                    insert_books(cur, chunk, author_ids)

                    print(f"Inserted {len(chunk)} authors and their books. Total valid authors so far: {valid_authors_count}")

                    if valid_authors_count >= MAX_AUTHORS:
                        print(f"Successfully validated {MAX_AUTHORS} authors.")
                        break

                if valid_authors_count < MIN_AUTHORS_REQUIRED:
                    print(f"Cannot proceed with data insertion. Found {valid_authors_count} valid authors, but {MIN_AUTHORS_REQUIRED} are required.")
                else:
                    print(f"Successfully inserted {valid_authors_count} valid authors.")

    except Exception as e:
        print(f"An error occurred during data insertion: {e}")

def insert_author(cur, author):
    """Inserts a single author into the database."""
    try:
        insert_author_query = """
            INSERT INTO library_author (name, image_url, bio)
            VALUES (%s, %s, %s)
            ON CONFLICT (name) DO NOTHING RETURNING id;
        """
        cur.execute(insert_author_query, (author["name"][:200], author["image_url"][:200], author["about"][:200]))
        
        if cur.rowcount > 0:
            print(f"Inserted author: {author['name']}")
            return cur.fetchone()[0]  # Return the new author ID
        
        print(f"Author already exists: {author['name']}")
        return None  # If no row was inserted
    except Exception as e:
        print(f"Error inserting author {author['name']}: {e}")
        return None

def insert_books(cur, chunk, author_ids):
    """Inserts books associated with authors into the database."""
    insert_book_query = """
        INSERT INTO library_book (title, description, author_id, published_date)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (title, author_id) DO NOTHING;
    """
    for author in chunk:
        author_id = author_ids.get(author["name"])
        if author_id is None:
            print(f"Author ID not found for {author['name']}.")
            continue

        for book in author.get("books_written", []):
            # Ensure the book is a string and not empty
            if isinstance(book, str) and book.strip():
                book_title = book.strip()  # Clean the book title
                published_date = "1900-01-01"  # Default date if not provided

                # Validate that the book title is not empty
                if book_title:
                    try:
                        cur.execute(
                            insert_book_query,
                            (
                                book_title,
                                "",  # Assuming description can be empty or fetch from author data
                                author_id,
                                published_date,
                            ),
                        )
                    except Exception as e:
                        print(f"Error inserting book '{book_title}' for author '{author['name']}': {e}")
                else:
                    print(f"Invalid book title for author {author['name']}: {book}")
            else:
                print(f"Invalid book data for author {author['name']}: {book}")

def process_authors(file_path):
    """Processes authors from the CSV and inserts valid data into the database."""
    authors_data = read_authors_from_csv(file_path)

    valid_authors = [author for author in authors_data if validate_author_data(author)][:MAX_AUTHORS]

    print(f"Processed {len(valid_authors)} valid authors, limited to {MAX_AUTHORS}.")

    if len(valid_authors) == MAX_AUTHORS:
        insert_data_in_chunks(valid_authors)
        print(f"🎉 Successfully validated and added {MAX_AUTHORS} authors to the database!")
    else:
        print(f"Cannot proceed with data insertion. Found {len(valid_authors)} valid authors, but {MAX_AUTHORS} are required.")

# Execute the script
if __name__ == "__main__":
    csv_file_path = "authors_filtered.csv"
    process_authors(csv_file_path)


File not found: authors_filtered.csv
Processed 0 valid authors, limited to 250.
Cannot proceed with data insertion. Found 0 valid authors, but 250 are required.
